# Funniness Estimation System  v4.0

In [ ]:
"""
@author: Ziyang Lin
         zlin19@sheffield.ac.uk
         University of Sheffield, UK
"""

'''
A system for
"Assessing the Funniness of Edited News Headlines (SemEval-2020)" task 2.
'''

import random

import pandas as pd
import numpy as np

import os
import re
import time
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext import data
from torchtext import datasets
import torch.utils.data as tud

from google.colab import drive 
drive.mount('/content/gdrive')

import nltk
nltk.download('punkt')
from nltk import word_tokenize


# fix the seeds to get consistent results before every training
# loop in what follows
def fix_seed(seed=1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


# Helper function to print the run time
def run_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
log_num_r = 0
real_task_stats = []

def add_real_task_stats(real_task_stats, log_num_r, MODEL_NAME, BATCH_SIZE, N_EPOCHS, LRATE, FRATE, EPS, WU, WDECAY, train_loss, train_accuracy, val_loss, val_accuracy, test_accuracy):
    log_num_r += 1

    real_task_stats.append(
        {
            'log': log_num_r,
            'Model Name': MODEL_NAME,
            'Batch Size': BATCH_SIZE,
            'N_Epochs': N_EPOCHS,
            'lr': LRATE,
            'fr': FRATE,
            'eps': EPS,
            'wu': WU,
            'wd': WDECAY,
            'Training Loss': train_loss,
            'Training Accur.': train_accuracy,
            'Valid. Loss': val_loss,
            'Valid. Accur.': val_accuracy,
            'Testing Accur.': test_accuracy
        }
    )
    
    return real_task_stats, log_num_r

In [ ]:
log_num_f = 0
fake_task_stats = []

def add_fake_task_stats(fake_task_stats, log_num_f, MODEL_NAME, BATCH_SIZE, N_EPOCHS, LRATE, FRATE, EPS, WU, WDECAY, train_loss, val_loss):
    log_num_f += 1

    fake_task_stats.append(
        {
            'log': log_num_f,
            'Model Name': MODEL_NAME,
            'Batch Size': BATCH_SIZE,
            'N_Epochs': N_EPOCHS,
            'lr': LRATE,
            'fr': FRATE,
            'eps': EPS,
            'wu': WU,
            'wd': WDECAY,
            'Training Loss': train_loss,
            'Valid. Loss': val_loss
        }
    )
    
    return fake_task_stats, log_num_f

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 4.8MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 3.0MB 45.7MB/s 
     |████████████████████████████████| 890kB 33.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=8793f0e139ddd519d3ee83c3e12e9fa34b278413e1a694d2807b8ca6fb99580a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# do computation on a GPU if possible 
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True
  DEVICE='cuda:0'
else:
  DEVICE='cpu'

print('Device is', DEVICE)

Device is cpu


# Preprocessing Datasets

## Read data from csv files

In [ ]:
train_loc = 'gdrive/My Drive/subtask-2/train.csv'
dev_loc = 'gdrive/My Drive/subtask-2/dev.csv'
test_loc = 'gdrive/My Drive/subtask-2/test.csv'
train = pd.read_csv(train_loc)  
valid = pd.read_csv(dev_loc)
test = pd.read_csv(test_loc)

In [ ]:
def get_edited_headlines_list(headls_words):
    # list of new edited headlines
    headls_list = []
    
    for origin_headl, new_word in headls_words:
      # pattern
      p = re.compile(r'\<(.*?)\/\>')
      # get the new edited headline
      new_headl = p.sub(new_word, origin_headl)
      # add it to the list
      headls_list.append(new_headl)

    return headls_list


def processed_data_to_lists(train):
    headls_words_1 = [(origin_headl_1, new_word_1) for (origin_headl_1, new_word_1) in zip(train.original1.to_list(), train.edit1.to_list())]
    headls_words_2 = [(origin_headl_2, new_word_2) for (origin_headl_2, new_word_2) in zip(train.original2.to_list(), train.edit2.to_list())]
    labels_list = train.label.to_list()
    meanGrade1_list = train.meanGrade1.to_list()
    meanGrade2_list = train.meanGrade2.to_list()
    meanGrade_list = meanGrade1_list + meanGrade2_list

    headls_1 = get_edited_headlines_list(headls_words_1)
    headls_2 = get_edited_headlines_list(headls_words_2)

    return headls_1, headls_2, labels_list, meanGrade_list



## Get lists of headlines and list of labels

In [ ]:
train_headls_1, train_headls_2, train_labels_list, train_meanGrade_list = processed_data_to_lists(train)
valid_headls_1, valid_headls_2, valid_labels_list, valid_meanGrade_list = processed_data_to_lists(valid)
test_headls_1, test_headls_2, test_labels_list, test_meanGrade_list = processed_data_to_lists(test)

In [ ]:
train_headls = train_headls_1 + train_headls_2
valid_headls = valid_headls_1 + valid_headls_2
test_headls = test_headls_1 + test_headls_2

In [ ]:
from transformers import AlbertTokenizer

# Load the ALBERT tokenizer.
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

In [ ]:
print(' Original: ', train_headls_1[0])

print('Tokenized: ', tokenizer.tokenize(train_headls_1[0]))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_headls_1[0])))

 Original:  " Gene Cernan , Last Dancer on the Moon , Dies at 82 "
Tokenized:  ['▁', '"', '▁gene', '▁cer', 'nan', '▁', ',', '▁last', '▁dancer', '▁on', '▁the', '▁moon', '▁', ',', '▁dies', '▁at', '▁82', '▁', '"']
Token IDs:  [13, 7, 3194, 6173, 3198, 13, 15, 236, 7799, 27, 14, 2121, 13, 15, 8972, 35, 9766, 13, 7]


## Max sequence length for ALBERT

In [ ]:
max_one_len = 0

for headl in train_headls:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(headl, add_special_tokens=True)
    # Update the maximum sentence length.
    max_one_len = max(max_one_len, len(input_ids))

print('Max sequence length for two sentences: ', (max_one_len-1)*2)
print('Max sequence length for one sentence: ', max_one_len)

Max sequence length for two sentences:  92
Max sequence length for one sentence:  47


## Get encoded inputs for ALBERT

In [ ]:
# prepare encoded inputs for real task
train_encoded_inputs = tokenizer(train_headls_1, train_headls_2, padding='max_length', max_length=92, truncation=True, return_tensors="pt")
valid_encoded_inputs = tokenizer(valid_headls_1, valid_headls_2, padding='max_length', max_length=92, truncation=True, return_tensors="pt")
test_encoded_inputs = tokenizer(test_headls_1, test_headls_2, padding='max_length', max_length=92, truncation=True, return_tensors="pt")

# prepare encoded inputs for fake task
pre_train_encoded = tokenizer(train_headls, padding='max_length', max_length=47, truncation=True, return_tensors="pt")
pre_valid_encoded = tokenizer(valid_headls, padding='max_length', max_length=47, truncation=True, return_tensors="pt")

In [ ]:
# get input_ids, attention_mask, token_type_ids and labels for real task
train_input_ids = train_encoded_inputs['input_ids']
train_attention_mask = train_encoded_inputs['attention_mask']
train_token_type_ids = train_encoded_inputs['token_type_ids']
train_labels = torch.tensor(train_labels_list)

valid_input_ids = valid_encoded_inputs['input_ids']
valid_attention_mask = valid_encoded_inputs['attention_mask']
valid_token_type_ids = valid_encoded_inputs['token_type_ids']
valid_labels = torch.tensor(valid_labels_list)

test_input_ids = test_encoded_inputs['input_ids']
test_attention_mask = test_encoded_inputs['attention_mask']
test_token_type_ids = test_encoded_inputs['token_type_ids']
test_labels = torch.tensor(test_labels_list)


# get input_ids, attention_mask and labels for fake task
pre_train_input_ids = pre_train_encoded['input_ids']
pre_train_attention_mask = pre_train_encoded['attention_mask']
pre_train_labels = torch.tensor(train_meanGrade_list)

pre_valid_input_ids = pre_valid_encoded['input_ids']
pre_valid_attention_mask = pre_valid_encoded['attention_mask']
pre_valid_labels = torch.tensor(valid_meanGrade_list)

pre_train_input_ids[0]
pre_train_labels

tensor([1.2000, 0.6000, 0.6000,  ..., 2.2000, 1.8000, 0.2000])

## Prepare mini-batches

In [ ]:
class BERT_Dataset(tud.Dataset):
    def __init__(self, x1, x2, x3, y1):
        self.len = x1.shape[0]

        self.x1_data = x1.to(DEVICE)
        self.x2_data = x2.to(DEVICE)
        self.x3_data = x3.to(DEVICE)
        self.y1_data = y1.to(DEVICE)


    def __getitem__(self, index):
        return self.x1_data[index], self.x2_data[index], self.x3_data[index], self.y1_data[index]


    def __len__(self):
        return self.len


class Pre_BERT_Dataset(tud.Dataset):
    def __init__(self, x1, x2, y1):
        self.len = x1.shape[0]

        self.x1_data = x1.to(DEVICE)
        self.x2_data = x2.to(DEVICE)
        self.y1_data = y1.to(DEVICE)


    def __getitem__(self, index):
        return self.x1_data[index], self.x2_data[index], self.y1_data[index]


    def __len__(self):
        return self.len

In [ ]:
fix_seed()
# Batching for BERT
BATCH_SIZE = 16

# For real task
train_dataset = BERT_Dataset(train_input_ids, train_attention_mask, train_token_type_ids, train_labels)
valid_dataset = BERT_Dataset(valid_input_ids, valid_attention_mask, valid_token_type_ids, valid_labels)
test_dataset = BERT_Dataset(test_input_ids, test_attention_mask, test_token_type_ids, test_labels)

train_dataloader = tud.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = tud.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = tud.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# For fake task
pre_train_dataset = Pre_BERT_Dataset(pre_train_input_ids, pre_train_attention_mask, pre_train_labels)
pre_valid_dataset = Pre_BERT_Dataset(pre_valid_input_ids, pre_valid_attention_mask, pre_valid_labels)

pre_train_dataloader = tud.DataLoader(pre_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
pre_valid_dataloader = tud.DataLoader(pre_valid_dataset, batch_size=BATCH_SIZE, shuffle=True)


##### demo #####
print(train_dataloader)

for x1, x2, x3, y1 in train_dataloader:
    demo_x1 = x1
    demo_x2 = x2
    demo_x3 = x3
    demo_y1 = y1
    break
    
print(x1.shape)
print(x2.shape)
print(x3.shape)
print(y1.shape)
print(len(train_dataloader))

torch.Size([16, 92])
torch.Size([16, 92])
torch.Size([16, 92])
torch.Size([16])
587


# Training Preparation 

## Define accuracy

In [ ]:
def accuracy(preds, y):
    """
    returns accuracy per batch
    """

    class_preds =  torch.argmax(F.softmax(preds, dim = 1), 1)
    correct = (class_preds == y).float() # convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

## Define train and evaluate

In [ ]:
# define train_BERT and evaluate_BERT for real task
def train_BERT(model, train_dataloader, valid_dataloader, optimizer, scheduler, criterion, N_EPOCHS):
    fix_seed()
    model = model.to(DEVICE)
    model_list = []
    # Measure the total time for the whole run.
    t0 = time.time()

    for epoch in range(N_EPOCHS):
    
        start_time = time.time()
        # To ensure the dropout is "turned on" while training
        model.train()
        
        epoch_loss = 0
        epoch_acc = 0
    
        for input_ids_batch, attention_mask_batch, token_type_ids_batch, labels in train_dataloader:
                        
            # Zero the gradients
            optimizer.zero_grad()
            # shape(input_ids_batch) = [B, T]
            # shape(attention_mask_batch) = [B, T]
            # shape(labels) = [B]

            # get the output
            predictions = model(input_ids_batch,
                                attention_mask_batch,
                                token_type_ids_batch)
            
            # calculate the loss
            loss = criterion(predictions.view(-1, 3), labels.view(-1))

            # calculate training accuracy
            acc = accuracy(predictions, labels)
                      
            # calculate the gradient of each parameter
            loss.backward()
        
            # update the parameters using the gradients and optimizer algorithm 
            optimizer.step()

            # update the learning rate
            scheduler.step()
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        average_epoch_loss = epoch_loss / len(train_dataloader)
        average_epoch_acc = epoch_acc / len(train_dataloader)
        
        end_time = time.time()
               
        epoch_mins, epoch_secs = run_time(start_time, end_time)
    
        average_epoch_valid_loss, average_epoch_valid_acc = evaluate_BERT(model, criterion, valid_dataloader)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {average_epoch_loss:.5f} | Train Acc: {average_epoch_acc*100:.4f}%')
        print(f'\t Val. Loss: {average_epoch_valid_loss:.5f} |  Val. Acc: {average_epoch_valid_acc*100:.4f}%')

        model_list.append(model)

    print("")
    print("***Completed***")
    total_mins, total_secs = run_time(t0, time.time())
    print(f'Total time spent: {total_mins}m {total_secs}s')

    return average_epoch_loss, average_epoch_acc, average_epoch_valid_loss, average_epoch_valid_acc, model_list

def evaluate_BERT(model, criterion, dataloader):
    fix_seed()
    epoch_loss = 0
    epoch_acc = 0
    # Turn on evaluate mode. This de-activates dropout. 
    model.eval()

    # We do not compute gradients within this block, i.e. no training
    with torch.no_grad():

        for input_ids_batch, attention_mask_batch, token_type_ids_batch, labels in dataloader:
            
            # get the output
            predictions = model(input_ids_batch,
                            attention_mask_batch,
                            token_type_ids_batch)

            loss = criterion(predictions.view(-1, 3), labels.view(-1))
            acc = accuracy(predictions, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [ ]:
# define train_BERT and evaluate_BERT for fake regression task
def pre_train_BERT(model, train_dataloader, valid_dataloader, optimizer, scheduler, criterion, N_EPOCHS):
    fix_seed()
    model = model.to(DEVICE)
    model_list = []
    t0 = time.time()

    for epoch in range(N_EPOCHS):    
        start_time = time.time()
        model.train()       
        epoch_loss = 0
        epoch_acc = 0
    
        for input_ids_batch, attention_mask_batch, labels in train_dataloader:                       
            optimizer.zero_grad()

            predictions = model(input_ids_batch,
                                attention_mask_batch)
            
            loss = criterion(predictions.view(-1), labels.view(-1))                      
            loss.backward()        
            optimizer.step()
            scheduler.step()          
            epoch_loss += loss.item()
            
        average_epoch_loss = epoch_loss / len(train_dataloader)
        
        end_time = time.time()               
        epoch_mins, epoch_secs = run_time(start_time, end_time)
    
        average_epoch_valid_loss = pre_evaluate_BERT(model, criterion, valid_dataloader)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {average_epoch_loss:.5f} ')
        print(f'\t Val. Loss: {average_epoch_valid_loss:.5f} ')
        
        model_list.append(model)

    print("")
    print("***Completed***")
    total_mins, total_secs = run_time(t0, time.time())
    print(f'Total time spent: {total_mins}m {total_secs}s')

    return average_epoch_loss, average_epoch_valid_loss, model_list

def pre_evaluate_BERT(model, criterion, dataloader):
    fix_seed()
    epoch_loss = 0
    model.eval()

    with torch.no_grad():

        for input_ids_batch, attention_mask_batch, labels in dataloader:
            
            predictions = model(input_ids_batch,
                                attention_mask_batch)

            loss = criterion(predictions.view(-1), labels.view(-1))
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

## Define models

In [ ]:
from transformers import AlbertModel, BertModel, ElectraModel, AdamW

class AlbertModel_FakeTask(nn.Module):
    def __init__(self):
        super(AlbertModel_FakeTask, self).__init__()
        self.albert = AlbertModel.from_pretrained('albert-base-v2')
        self.linear = nn.Linear(768, 1)

    def forward(self, source, mask):
        output = self.albert(source, attention_mask=mask)
        # take the last hidden state
        hidden = output[0]
        # take the representations for CLS
        all_cls = hidden[:, 0, :]
        # pass to linear layer to get the score for each sentence(cls here)
        pred = self.linear(all_cls)

        return pred


class AlbertModel_Real(nn.Module):
    def __init__(self, albert_model):
        super(AlbertModel_Real, self).__init__()
        self.albert = albert_model
        self.classifier = nn.Linear(768, 3)

    def forward(self, source, mask, type_ids):
        output = self.albert(source, attention_mask=mask, token_type_ids=type_ids)
        # take the last hidden state
        hidden = output[0]
        # take the representations for CLS
        all_cls = hidden[:, 0, :]
        # pass to linear layer to get the prediction for every sentence pairs(cls here)
        pred = self.classifier(all_cls)

        return pred

# Fake Task

In [ ]:
fix_seed()
# Create the pre_model for the fake task
pre_model = AlbertModel_FakeTask()

param_names_list = [n for n, p in pre_model.named_parameters()]
param_names_list

['albert.embeddings.word_embeddings.weight',
 'albert.embeddings.position_embeddings.weight',
 'albert.embeddings.token_type_embeddings.weight',
 'albert.embeddings.LayerNorm.weight',
 'albert.embeddings.LayerNorm.bias',
 'albert.encoder.embedding_hidden_mapping_in.weight',
 'albert.encoder.embedding_hidden_mapping_in.bias',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key.bias',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight',
 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias',
 'albert.encoder.albert_layer_gro

## Hyperparameters

In [ ]:
# Hyperparameters for the fake task LM:

# Number of training epochs. The BERT authors recommend between 2 and 4.
N_EPOCHS = 3

LRATE = 5e-3
FRATE = 2e-5
EPS = 1e-8
WU = 0.1
WDECAY = 0.01

# Total number of training steps is [number of batches] x [number of epochs]. 
TOTSTEPS = len(train_dataloader) * N_EPOCHS
WUSTEPS = int(TOTSTEPS * WU)

# Apply weight decay to all parameters other than bias and layer normalization terms
# Optimize the parameters of the head layer by the learning rate
# Optimize the parameters of the pretrain LM by the fine-tuning rate
no_decay = ['bias', 'LayerNorm.weight']
named_parameters = pre_model.named_parameters()
optimizer_grouped_parameters = [
    {'params': [p for n, p in named_parameters if not any(nd in n for nd in no_decay)], 'weight_decay': WDECAY},
    {'params': [p for n, p in named_parameters if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in named_parameters if "albert" not in n], 'lr': LRATE}
]
#    {'params': [p for n, p in named_parameters if "albert" in n], 'lr': FRATE},

## Optimizer & learning rate scheduler

In [ ]:
# Create the optimizer, 
# the epsilon parameter is a very small number to prevent any division by zero
optimizer = AdamW(optimizer_grouped_parameters, lr=FRATE, eps = EPS)

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = WUSTEPS,
                                            num_training_steps = TOTSTEPS)

## Define RMSE

In [ ]:
# define rmse
def rmse(predictions, labels):
    loss = torch.sqrt(((predictions - labels)**2).mean())

    return loss

## Fake task training

In [ ]:
criterion_r = rmse

train_loss, val_loss, model_list = pre_train_BERT(pre_model,
                                                  pre_train_dataloader,
                                                  pre_valid_dataloader,
                                                  optimizer,
                                                  scheduler,
                                                  criterion_r,
                                                  N_EPOCHS)

Epoch: 01 | Epoch Time: 2m 56s
	Train Loss: 0.56912 
	 Val. Loss: 0.53872 
Epoch: 02 | Epoch Time: 2m 55s
	Train Loss: 0.52356 
	 Val. Loss: 0.53316 
Epoch: 03 | Epoch Time: 2m 54s
	Train Loss: 0.51733 
	 Val. Loss: 0.53316 

***Completed***
Total time spent: 9m 38s


## Fake task logging

In [ ]:
B1 = "bert-base-uncased"
A2 = "albert-base-v2"
E = "electra"

fake_task_stats, log_num_f = add_fake_task_stats(fake_task_stats, 
                                             log_num_f,
                                             A2,
                                             BATCH_SIZE, 
                                             N_EPOCHS,
                                             "{:.0e}".format(LRATE),
                                             "{:.0e}".format(FRATE), 
                                             "{:.0e}".format(EPS), 
                                             WU,
                                             WDECAY, 
                                             train_loss,
                                             val_loss
                                             )

pd.set_option('precision', 5)
df_stats = pd.DataFrame(data=fake_task_stats)
df_stats = df_stats.set_index('log')
# Display the table.
df_stats

,Model Name,Batch Size,N_Epochs,lr,fr,eps,wu,wd,Training Loss,Valid. Loss
log,,,,,,,,,,
1,albert-base-v2,16,3,5e-03,2e-05,1e-08,0.1,0.01,0.57585,0.57040
2,albert-base-v2,16,3,5e-03,2e-05,1e-08,0.1,0.01,0.57590,0.57045
3,albert-base-v2,16,3,5e-03,2e-05,1e-08,0.1,0.01,0.51733,0.53316


In [ ]:
log_loc = 'gdrive/My Drive/subtask-2/log_fake_task.csv'
df_stats.to_csv(log_loc, index=False)

# Real Task

In [ ]:
fix_seed()
# Create the model for the real task
albert_model = pre_model.albert
real_model = AlbertModel_Real(albert_model)

param_names_list = [n for n, p in real_model.named_parameters()]
param_names_list

## Hyperparameters

In [ ]:
# Hyperparameters for the real task model:

# Number of training epochs. The BERT authors recommend between 2 and 4.
N_EPOCHS_r = 3

LRATE_r = 3e-5
FRATE_r = 5e-5
EPS_r = 1e-8
WU_r = 0.1
WDECAY_r = 0.01

# Total number of training steps is [number of batches] x [number of epochs]. 
TOTSTEPS_r = len(train_dataloader) * N_EPOCHS_r
WUSTEPS_r = int(TOTSTEPS_r * WU_r)

# Apply weight decay to all parameters other than bias and layer normalization terms
# Optimize the parameters of the head layer by the learning rate
# Optimize the parameters of the pretrain LM by the fine-tuning rate
no_decay_r = ['bias', 'LayerNorm.weight']
named_parameters_r = real_model.named_parameters()
optimizer_grouped_parameters_r = [
    {'params': [p for n, p in named_parameters_r if not any(nd in n for nd in no_decay_r)], 'weight_decay': WDECAY_r},
    {'params': [p for n, p in named_parameters_r if any(nd in n for nd in no_decay_r)], 'weight_decay': 0.0}
]

## Optimizer & learning rate scheduler

In [ ]:
# Create the optimizer, 
# the epsilon parameter is a very small number to prevent any division by zero
optimizer_r = AdamW(optimizer_grouped_parameters_r, lr=LRATE_r, eps = EPS_r)

In [ ]:
# Create the learning rate scheduler.
scheduler_r = get_linear_schedule_with_warmup(optimizer_r, 
                                            num_warmup_steps = WUSTEPS_r,
                                            num_training_steps = TOTSTEPS_r)

## Real task training

In [ ]:
criterion_c = nn.CrossEntropyLoss()

train_loss_r, train_accuracy_r, val_loss_r, val_accuracy_r, model_list_r = train_BERT(real_model,
                                                                            train_dataloader,
                                                                            valid_dataloader,
                                                                            optimizer_r,
                                                                            scheduler_r,
                                                                            criterion_c,
                                                                            N_EPOCHS_r)


Epoch: 01 | Epoch Time: 2m 48s
	Train Loss: 0.96326 | Train Acc: 45.9263%
	 Val. Loss: 0.97672 |  Val. Acc: 43.0743%
Epoch: 02 | Epoch Time: 2m 46s
	Train Loss: 0.96141 | Train Acc: 44.6359%
	 Val. Loss: 0.95989 |  Val. Acc: 46.1149%
Epoch: 03 | Epoch Time: 2m 46s
	Train Loss: 0.95841 | Train Acc: 45.5324%
	 Val. Loss: 0.96040 |  Val. Acc: 45.7348%

***Completed***
Total time spent: 9m 8s


# Testing Real Task Models

## Start testing

In [ ]:
fix_seed()

test_loss = 0
test_acc = 0
test_logits_all = []

# Turn on evaluate mode. This de-activates dropout. 
real_model.eval()

# We do not compute gradients within this block, i.e. no training
with torch.no_grad():

    for input_ids_batch, attention_mask_batch, token_type_ids_batch, labels in test_dataloader:
        
        # get the output
        predictions = real_model(input_ids_batch,
                                 attention_mask_batch,
                                 token_type_ids_batch)

        loss_batch = criterion_c(predictions, labels)
        test_logits_all += predictions.tolist()
        acc_batch = accuracy(predictions, labels)

        test_loss += loss_batch.item()
        test_acc += acc_batch.item()

    average_test_loss = test_loss / len(test_dataloader)
    average_test_acc = test_acc / len(test_dataloader)

print(f'Test Loss: {average_test_loss:.5f} | Test Acc: {average_test_acc*100:.7f}%')

Test Loss: 0.96650 | Test Acc: 45.2364865%


## Write results

In [ ]:
def write_predictions(predictions, test_data_frame, out_loc):
    test_data_frame['pred'] = predictions
    output = test_data_frame[['id','pred']]
    output.to_csv(out_loc, index=False)
        
    print('Output file created:\n\t- '+os.path.abspath(out_loc))


# write the predictions for the dev data into 'task-2-output.csv'
out_loc = 'gdrive/My Drive/subtask-2/task-2-output.csv'

test_class_preds = torch.argmax(F.softmax(torch.FloatTensor(test_logits_all), dim = 1), 1)
write_predictions(test_class_preds, test, out_loc)

Output file created:
	- /content/gdrive/My Drive/subtask-2/task-2-output.csv


## Check final results

In [ ]:
def score(truth_loc, prediction_loc):
    truth = pd.read_csv(truth_loc, usecols=['id','label'])
    pred = pd.read_csv(prediction_loc, usecols=['id','pred'])
         
    assert(sorted(truth.id) == sorted(pred.id)),"ID mismatch between ground truth and prediction!"
    
    data = pd.merge(truth,pred)
    data = data[data.label != 0]
    accuracy = (np.sum(data.label == data.pred)*1.0/len(data))*100
    
    print("Accuracy = %.6f" % accuracy)

    return accuracy   

# print Accuracy
truth_loc = 'gdrive/My Drive/subtask-2/test.csv'
prediction_loc = 'gdrive/My Drive/subtask-2/task-2-output.csv'
test_accuracy = score(truth_loc, prediction_loc)

Accuracy = 50.951294


# Logging Real Task Statistics

In [ ]:
B1 = "bert-base-uncased"
A2 = "albert-base-v2"
E = "electra"

real_task_stats, log_num_r = add_real_task_stats(real_task_stats, 
                                             log_num_r,
                                             A2,
                                             BATCH_SIZE, 
                                             N_EPOCHS_r,
                                             "{:.0e}".format(LRATE_r),
                                             "{:.0e}".format(FRATE_r), 
                                             "{:.0e}".format(EPS_r), 
                                             WU_r,
                                             WDECAY_r, 
                                             train_loss_r, 
                                             train_accuracy_r*100, 
                                             val_loss_r, 
                                             val_accuracy_r*100,
                                             test_accuracy
                                             )

In [ ]:
# Display floats with five decimal places.
pd.set_option('precision', 5)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=real_task_stats)

# Use the 'log' as the row index.
df_stats = df_stats.set_index('log')

# Display the table.
df_stats

,Model Name,Batch Size,N_Epochs,lr,fr,eps,wu,wd,Training Loss,Training Accur.,Valid. Loss,Valid. Accur.,Testing Accur.
log,,,,,,,,,,,,,
1,albert-base-v2,16,3,5e-03,2e-05,1e-08,0.1,0.01,0.95910,44.78066,0.96082,43.07432,50.95129
2,albert-base-v2,16,3,5e-05,5e-05,1e-08,0.1,0.01,0.96709,44.35477,0.96287,43.03209,50.95129


## Write statistics

In [ ]:
log_loc = 'gdrive/My Drive/subtask-2/log2.csv'
df_stats.to_csv(log_loc, index=False)

In [ ]:
# remove the last row:
#training_stats.pop(-1)
#log_num -= 1